# THE PART 3 is below 

# PART 1

In this first part of Notebook the objective is use Wikipedia Info for create a table with Postal Codes from URL. 

In [5]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from geopy.geocoders import Nominatim

from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [6]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urlopen(url).read().decode('utf-8')
soup = BeautifulSoup(page, 'html.parser')

wiki_table = soup.body.table.tbody

In [7]:

def get_cell(element):
    cells = element.find_all('td')
    row = []
    
    for cell in cells:
        if cell.a:            
            if (cell.a.text):
                row.append(cell.a.text)
                continue
        row.append(cell.string.strip())
        
    return row

In [8]:
def get_row():    
    data = []  
    
    for tr in wiki_table.find_all('tr'):
        row = get_cell(tr)
        if len(row) != 3:
            continue
        data.append(row)        
    
    return data

In [9]:
# To get 3 Columns : PostalCode , Borough and Neighbourhood

data = get_row()
columns = ['PostalCode', 'Borough', 'Neighbourhood']
df = pd.DataFrame(data, columns=columns)

df.head()

PostalCode           Borough              Neighbourhood
0        M1A      Not assigned               Not assigned
1        M2A      Not assigned               Not assigned
2        M3A        North York                  Parkwoods
3        M4A        North York           Victoria Village
4        M5A  Downtown Toronto  Regent Park, Harbourfront

In [10]:
#Shape before procesing "not assigned"

df.shape

(180, 3)

In [11]:
# Processing 

df1 = df[df.Borough != 'Not assigned']
df1 = df1.sort_values(by=['PostalCode','Borough'])

df1.reset_index(inplace=True)
df1.drop('index',axis=1,inplace=True)

df1.head()

PostalCode      Borough                           Neighbourhood
0        M1B  Scarborough                          Malvern, Rouge
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2        M1E  Scarborough       Guildwood, Morningside, West Hill
3        M1G  Scarborough                                  Woburn
4        M1H  Scarborough                               Cedarbrae

In the above table, we have an only PostalCode for different Neighbourdhood

In [12]:
df_postcodes = df1['PostalCode']
df_postcodes.drop_duplicates(inplace=True)
df2 = pd.DataFrame(df_postcodes)
df2['Borough'] = '';
df2['Neighbourhood'] = '';


df2.reset_index(inplace=True)
df2.drop('index', axis=1, inplace=True)
df1.reset_index(inplace=True)
df1.drop('index', axis=1, inplace=True)

for i in df2.index:
    for j in df1.index:
        if df2.iloc[i, 0] == df1.iloc[j, 0]:
            df2.iloc[i, 1] = df1.iloc[j, 1]
            df2.iloc[i, 2] = df2.iloc[i, 2] + ',' + df1.iloc[j, 2]
            
for i in df2.index:
    s = df2.iloc[i, 2]
    if s[0] == ',':
        s =s [1:]
    df2.iloc[i,2 ] = s

In [13]:
df2.shape

(103, 3)

The rows of this DF is 103 with 3 columns, the next assignment is other Notebook.

# PART 2 

In this part I use the .csv file with the geographical coordinates and merge with the file of Part 1. 

In [14]:
df = pd.read_csv("https://cocl.us/Geospatial_data")
df

Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
5           M1J  43.744734 -79.239476
6           M1K  43.727929 -79.262029
7           M1L  43.711112 -79.284577
8           M1M  43.716316 -79.239476
9           M1N  43.692657 -79.264848
10          M1P  43.757410 -79.273304
11          M1R  43.750071 -79.295849
12          M1S  43.794200 -79.262029
13          M1T  43.781638 -79.304302
14          M1V  43.815252 -79.284577
15          M1W  43.799525 -79.318389
16          M1X  43.836125 -79.205636
17          M2H  43.803762 -79.363452
18          M2J  43.778517 -79.346556
19          M2K  43.786947 -79.385975
20          M2L  43.757490 -79.374714
21          M2M  43.789053 -79.408493
22          M2N  43.770120 -79.408493
23          M2P  43.752758 -79.400049
24          M2R  43.782736 -79.442259
25          M3A  43.753259 -79.329656
26          M3B  43.745906 -79.352188
27          M3C  43.725900 -79.340923
28          M3H  43.754328 -79.442259
29          M3J  43.767980 -79.487262
30          M3K  43.737473 -79.464763
31          M3L  43.739015 -79.506944
32          M3M  43.728496 -79.495697
33          M3N  43.761631 -79.520999
34          M4A  43.725882 -79.315572
35          M4B  43.706397 -79.309937
36          M4C  43.695344 -79.318389
37          M4E  43.676357 -79.293031
38          M4G  43.709060 -79.363452
39          M4H  43.705369 -79.349372
40          M4J  43.685347 -79.338106
41          M4K  43.679557 -79.352188
42          M4L  43.668999 -79.315572
43          M4M  43.659526 -79.340923
44          M4N  43.728020 -79.388790
45          M4P  43.712751 -79.390197
46          M4R  43.715383 -79.405678
47          M4S  43.704324 -79.388790
48          M4T  43.689574 -79.383160
49          M4V  43.686412 -79.400049
50          M4W  43.679563 -79.377529
51          M4X  43.667967 -79.367675
52          M4Y  43.665860 -79.383160
53          M5A  43.654260 -79.360636
54          M5B  43.657162 -79.378937
55          M5C  43.651494 -79.375418
56          M5E  43.644771 -79.373306
57          M5G  43.657952 -79.387383
58          M5H  43.650571 -79.384568
59          M5J  43.640816 -79.381752
60          M5K  43.647177 -79.381576
61          M5L  43.648198 -79.379817
62          M5M  43.733283 -79.419750
63          M5N  43.711695 -79.416936
64          M5P  43.696948 -79.411307
65          M5R  43.672710 -79.405678
66          M5S  43.662696 -79.400049
67          M5T  43.653206 -79.400049
68          M5V  43.628947 -79.394420
69          M5W  43.646435 -79.374846
70          M5X  43.648429 -79.382280
71          M6A  43.718518 -79.464763
72          M6B  43.709577 -79.445073
73          M6C  43.693781 -79.428191
74          M6E  43.689026 -79.453512
75          M6G  43.669542 -79.422564
76          M6H  43.669005 -79.442259
77          M6J  43.647927 -79.419750
78          M6K  43.636847 -79.428191
79          M6L  43.713756 -79.490074
80          M6M  43.691116 -79.476013
81          M6N  43.673185 -79.487262
82          M6P  43.661608 -79.464763
83          M6R  43.648960 -79.456325
84          M6S  43.651571 -79.484450
85          M7A  43.662301 -79.389494
86          M7R  43.636966 -79.615819
87          M7Y  43.662744 -79.321558
88          M8V  43.605647 -79.501321
89          M8W  43.602414 -79.543484
90          M8X  43.653654 -79.506944
91          M8Y  43.636258 -79.498509
92          M8Z  43.628841 -79.520999
93          M9A  43.667856 -79.532242
94          M9B  43.650943 -79.554724
95          M9C  43.643515 -79.577201
96          M9L  43.756303 -79.565963
97          M9M  43.724766 -79.532242
98          M9N  43.706876 -79.518188
99          M9P  43.696319 -79.532242
100         M9R  43.688905 -79.554724
101         M9V  43.739416 -79.588437
102         M9W  43.706748 -79.594054

In [20]:
#The next step is merge with the previous df2. 

neighborhoods_lat_long = df2.merge(df, left_on='PostalCode', right_on='Postal Code')
toronto_neighborhoods = neighborhoods_lat_long.drop('Postal Code', axis=1)
toronto_neighborhoods.head()

PostalCode      Borough                           Neighbourhood   Latitude  \
0        M1B  Scarborough                          Malvern, Rouge  43.806686   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

In [21]:
toronto_neighborhoods.shape

(103, 5)

# PART 3 

In [22]:
# Calling different tools for this part

import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim #

import requests 
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

import folium 

I choose a subset of all the Borough that contain the word "Toronto" and ...

In [23]:
toronto_data= toronto_neighborhoods[toronto_neighborhoods['Borough'].str.contains('Toronto', na = False)].reset_index(drop=True)
toronto_data.head()

PostalCode          Borough                   Neighbourhood   Latitude  \
0        M4E     East Toronto                     The Beaches  43.676357   
1        M4K     East Toronto    The Danforth West, Riverdale  43.679557   
2        M4L     East Toronto  India Bazaar, The Beaches West  43.668999   
3        M4M     East Toronto                 Studio District  43.659526   
4        M4N  Central Toronto                   Lawrence Park  43.728020   

   Longitude  
0 -79.293031  
1 -79.352188  
2 -79.315572  
3 -79.340923  
4 -79.388790

In [30]:
print ("This subset has", toronto_data.shape,"rows and columns")

This subset has (39, 5) rows and columns


Toronto geographical location is :

In [31]:
latitude = 43.6532
longitude= -79.3832


A map for the previous dataset is ...

In [32]:

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

USING FOURSQUARE INFO

In [34]:
#Define values to connect with Foursquare

CLIENT_ID = 'JWEI3CY4QL3TFD5Y1JDJ2YUFEEJKZPRSONAEPVEBYSQ2ZOBZ' # your Foursquare ID
CLIENT_SECRET = 'ECT5PDDLPXNDXVWCG1QSSJETHHD1BFRAT0WBUHST5RYTPX04' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 500 

Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [38]:
print(toronto_venues.shape)
toronto_venues.head()

(1603, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  The Beaches              43.676357              -79.293031   
1  The Beaches              43.676357              -79.293031   
2  The Beaches              43.676357              -79.293031   
3  The Beaches              43.676357              -79.293031   
4  The Beaches              43.676357              -79.293031   

                                Venue  Venue Latitude  Venue Longitude  \
0                   Glen Manor Ravine       43.676821       -79.293942   
1  The Big Carrot Natural Food Market       43.678879       -79.297734   
2                 Grover Pub and Grub       43.679181       -79.297215   
3                       Upper Beaches       43.680563       -79.292869   
4                 Seaspray Restaurant       43.678888       -79.298167   

      Venue Category  
0              Trail  
1  Health Food Store  
2                Pub  
3       Neighborhood  
4   Asian Restaurant

In [39]:
toronto_venues.groupby('Neighborhood').count()


Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            56   
Brockton, Parkdale Village, Exhibition Place                           23   
Business reply mail Processing Centre, South Ce...                     15   
CN Tower, King and Spadina, Railway Lands, Harb...                     16   
Central Bay Street                                                     61   
Christie                                                               15   
Church and Wellesley                                                   77   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             34   
Davisville North                                                        8   
Dufferin, Dovercourt Village                                           17   
First Canadian Place, Underground city                                100   
Forest Hill North & West, Forest Hill Road Park                         4   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                     100   
High Park, The Junction South                                          24   
India Bazaar, The Beaches West                                         21   
Kensington Market, Chinatown, Grange Park                              61   
Lawrence Park                                                           4   
Little Portugal, Trinity                                               43   
Moore Park, Summerhill East                                             2   
North Toronto West,  Lawrence Park                                     20   
Parkdale, Roncesvalles                                                 14   
Queen's Park, Ontario Provincial Government                            35   
Regent Park, Harbourfront                                              45   
Richmond, Adelaide, King                                               96   
Rosedale                                                                4   
Roselawn                                                                2   
Runnymede, Swansea                                                     33   
St. James Town                                                         79   
St. James Town, Cabbagetown                                            44   
Stn A PO Boxes                                                         97   
Studio District                                                        38   
Summerhill West, Rathnelly, South Hill, Forest ...                     15   
The Annex, North Midtown, Yorkville                                    21   
The Beaches                                                             5   
The Danforth West, Riverdale                                           42   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         32   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             56   
Brockton, Parkdale Village, Exhibition Place                            23   
Business reply mail Processing Centre, South Ce...                      15   
CN Tower, King and Spadina, Railway Lands, Harb...                      16   
Central Bay Street                                                      61   
Christie                                                                15   
Church and Wellesley                                                    77   
Commerce Court, Victoria Hotel                                         100   
Davisville                                                              34   
Davisville North                    

In [41]:
#UNIQUE CATEGORIES

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))


There are 232 uniques categories.


It is possible to see the frequency of events in each neighbourdhood 

In [51]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols=list(toronto_onehot.columns.values)
cols.pop(cols.index('Neighborhood'))
toronto_onehot=toronto_onehot[['Neighborhood']+cols]

# rename Neighborhood for Neighbourhood so that future merge works
toronto_onehot.rename(columns = {'Neighborhood': 'Neighbourhood'}, inplace = True)
toronto_onehot.head()

Neighbourhood  Adult Boutique  Airport  Airport Food Court  Airport Gate  \
0   The Beaches               0        0                   0             0   
1   The Beaches               0        0                   0             0   
2   The Beaches               0        0                   0             0   
3   The Beaches               0        0                   0             0   
4   The Beaches               0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  Aquarium  Art Gallery  Art Museum  Arts & Crafts Store  \
0             0         0            0           0                    0   
1             0         0            0           0                    0   
2             0         0            0           0                    0   
3             0         0            0           0                    0   
4             0         0            0           0                    0   

   Asian Restaurant  Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  Bakery  \
0                 0              0          0           0           0       0   
1                 0              0          0           0           0       0   
2                 0              0          0           0           0       0   
3                 0              0          0           0           0       0   
4                 1              0          0           0           0       0   

   Bank  Bar  Baseball Stadium  Basketball Stadium  Beach  Bed & Breakfast  \
0     0    0                 0                   0      0                0   
1     0    0                 0                   0      0                0   
2     0    0                 0                   0      0                0   
3     0    0                 0                   0      0                0   
4     0    0                 0                   0      0                0   

   Beer Bar  Beer Store  Belgian Restaurant  Bistro  Boat or Ferry  Bookstore  \
0         0           0                   0       0              0          0   
1         0           0                   0       0              0          0   
2         0           0                   0       0              0          0   
3         0           0                   0       0              0          0   
4         0           0                   0       0              0          0   

   Boutique  Brazilian Restaurant  Breakfast Spot  Brewery  Bubble Tea Shop  \
0         0                     0               0        0                0   
1         0                     0               0        0                0   
2         0                     0               0        0                0   
3         0                     0               0        0                0   
4         0                     0               0        0                0   

   Building  Burger Joint  Burrito Place  Bus Line  Business Service  Butcher  \
0         0             0              0         0                 0        0   
1         0             0              0         0                 0        0   
2         0             0              0         0                 0        0   
3         0             0              0         0                 0        0   
4         0             0              0         0                 0        0   

   Café  Cajun / Creole Restaurant  Candy Store  Caribbean Restaurant  \
0     0                          0            0                     0   
1     0                          0            0                     

In [52]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()


In [53]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2      Farmers Market  0.04
3  Seafood Restaurant  0.04
4         Cheese Shop  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.13
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3             Gym  0.04
4          Bakery  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0           Pizza Place  0.07
1         Auto Workshop  0.07
2            Skate Park  0.07
3  Fast Food Restaurant  0.07
4         Burrito Place  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0     Airport Service  0.12
1    Airport Terminal  0.12
2               Plane  0.06
3       Boat or Ferry  0.06
4  Airport Food Court  0.06


----Central Bay Street----
                venue  f

In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [55]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

Neighbourhood 1st Most Common Venue  \
0                                         Berczy Park           Coffee Shop   
1        Brockton, Parkdale Village, Exhibition Place                  Café   
2   Business reply mail Processing Centre, South C...           Pizza Place   
3   CN Tower, King and Spadina, Railway Lands, Har...       Airport Service   
4                                  Central Bay Street           Coffee Shop   
5                                            Christie         Grocery Store   
6                                Church and Wellesley           Coffee Shop   
7                      Commerce Court, Victoria Hotel           Coffee Shop   
8                                          Davisville           Pizza Place   
9                                    Davisville North  Gym / Fitness Center   
10                       Dufferin, Dovercourt Village              Pharmacy   
11             First Canadian Place, Underground city           Coffee Shop   
12    Forest Hill North & West, Forest Hill Road Park    Mexican Restaurant   
13                           Garden District, Ryerson        Clothing Store   
14  Harbourfront East, Union Station, Toronto Islands           Coffee Shop   
15                      High Park, The Junction South    Mexican Restaurant   
16                     India Bazaar, The Beaches West  Fast Food Restaurant   
17          Kensington Market, Chinatown, Grange Park                  Café   
18                                      Lawrence Park                  Park   
19                           Little Portugal, Trinity                   Bar   
20                        Moore Park, Summerhill East            Playground   
21                 North Toronto West,  Lawrence Park           Coffee Shop   
22                             Parkdale, Roncesvalles             Gift Shop   
23        Queen's Park, Ontario Provincial Government           Coffee Shop   
24                          Regent Park, Harbourfront           Coffee Shop   
25                           Richmond, Adelaide, King           Coffee Shop   
26                                           Rosedale                  Park   
27                                           Roselawn          Home Service   
28                                 Runnymede, Swansea                  Café   
29                                     St. James Town           Coffee Shop   
30                        St. James Town, Cabbagetown           Coffee Shop   
31                                     Stn A PO Boxes           Coffee Shop   
32                                    Studio District           Coffee Shop   
33  Summerhill West, Rathnelly, South Hill, Forest...           Coffee Shop   
34                The Annex, North Midtown, Yorkville        Sandwich Place   
35                                        The Beaches                   Pub   
36                       The Danforth West, Riverdale      Greek Restaurant   
37           Toronto Dominion Centre, Design Exchange           Coffee Shop   
38                     University of Toronto, Harbord                  Café   

   2nd Most Common Venue 3rd Most Common Venue          4th Most Common Venue  \
0           Cocktail Bar        Farmers Market             Seafood Restaurant   
1         Breakfast Spot           Coffee Shop                            Gym   
2          Auto Workshop            Skate Park           Fast Food Restaurant   
3       Airport Terminal                 Plane                  Boat or Ferry   
4         Sandwich Place                  Café             Italian Restaurant   
5                   Café                  Park                     Baby Store   
6       Sushi Restaurant   Japanese Restaurant                     Restaurant   
7             Restaurant                 Hotel                           Café   
8           Dessert Shop        Sandwich Place                            Gym   
9                  Hotel                  Park               Departme

# Cluster analysis

In [56]:
# numbers of K 
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [57]:
toronto_merged = toronto_data

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

PostalCode          Borough                   Neighbourhood   Latitude  \
0        M4E     East Toronto                     The Beaches  43.676357   
1        M4K     East Toronto    The Danforth West, Riverdale  43.679557   
2        M4L     East Toronto  India Bazaar, The Beaches West  43.668999   
3        M4M     East Toronto                 Studio District  43.659526   
4        M4N  Central Toronto                   Lawrence Park  43.728020   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.293031               0                   Pub     Health Food Store   
1 -79.352188               0      Greek Restaurant           Coffee Shop   
2 -79.315572               0  Fast Food Restaurant                   Gym   
3 -79.340923               0           Coffee Shop                  Café   
4 -79.388790               0                  Park              Bus Line   

  3rd Most Common Venue   4th Most Common Venue 5th Most Common Venue  \
0      Asian Restaurant                   Trail        Adult Boutique   
1    Italian Restaurant  Furniture / Home Store        Ice Cream Shop   
2      Sushi Restaurant                    Park           Pizza Place   
3             Gastropub     American Restaurant               Brewery   
4      Business Service             Swim School        Adult Boutique   

  6th Most Common Venue     7th Most Common Venue  \
0   Moroccan Restaurant       Monument / Landmark   
1                Lounge   Fruit & Vegetable Store   
2           Coffee Shop                       Pub   
3                Bakery                       Bar   
4         Movie Theater  Mediterranean Restaurant   

             8th Most Common Venue       9th Most Common Venue  \
0  Molecular Gastronomy Restaurant  Modern European Restaurant   
1               Frozen Yogurt Shop                   Bookstore   
2                       Restaurant               Movie Theater   
3        Latin American Restaurant                     Gay Bar   
4                      Men's Store          Mexican Restaurant   

      10th Most Common Venue  
0  Middle Eastern Restaurant  
1                        Spa  
2             Sandwich Place  
3           Stationery Store  
4  Middle Eastern Restaurant

In [58]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters